# 第二节    神经网络实现分类任务

- 网络基本构建与训练方法，常用函数解析

- torch.nn.functional模块

- nn.Module模块

#### Mnist分类任务

1. 注意数据需转换成tensor才能参与后续建模训练
```python
import torch

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)
```
---
2. 如果模型有可学习的参数，最好用nn.Module，其他情况nn.functional相对更简单一些
```python
# nn.functional()
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb.mm(weights) + bias

bs = 64
xb = x_train[0:bs]  # a mini-batch from x
yb = y_train[0:bs]

weights = torch.randn([784, 10], dtype = torch.float,  requires_grad = True) 
bias = torch.zeros(10, requires_grad=True)

print(loss_func(model(xb), yb))

```
---

3. 创建一个model来更简化代码

- 必须继承nn.Module且在其构造函数中需调用nn.Module的构造函数
- 无需写反向传播函数，nn.Module能够利用autograd自动实现反向传播
- Module中的可学习参数可以通过named_parameters()或者parameters()返回迭代器

```python
# nn.Module()
from torch import nn

class Mnist_NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(748,128)
        self.hidden2 = nn.Linear(128,256)
        self.out = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = self.dropout(x)     # 全连接层后可以加上dropout
        x = F.relu(self.hidden2(x))
        x = self.dropout(x)
        x = self.out(x)
        return x

net = Mnist_NN()

# print(net)
# for name, parameter in net.named_parameters():
#     print(name, parameter, parameter.size())

```
---
4. 使用TensorDataset和DataLoader导入数据  
DataLoader用于数据“打包”
```python
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)    
# 训练集打乱顺序，shuffle = True

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )

```

---
5. 训练网络
- 一般在训练模型时加上model.train()，这样会正常使用Batch Normalization和 Dropout
- 测试的时候一般选择model.eval()，这样就不会使用Batch Normalization和 Dropout

```python
# 训练网络
import numpy as np
def fit(steps, model, loss_func, opt, train_dl, valid_dl):
    for step in range(steps):       
        model.train()
        for xb, yb in train_dl:
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        print('当前step:'+str(step), '验证集损失：'+str(val_loss))


from torch import optim

def get_model():
    model = Mnist_NN()
    return model, optim.Adam(model.parameters(), lr=0.001)


def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()     # 算反向梯度
        opt.step()          # 梯度下降优化
        opt.zero_grad()     # 注意还原

    return loss.item(), len(xb)


# 主程序：
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
model, opt = get_model()
fit(25, model, loss_func, opt, train_dl, valid_dl)

# epoch：整轮，训练一次整个训练集
# bitch：小批，每单次训练的数据个数

```

---

```python
# 准确率
correct = 0
total = 0
for xb, yb in valid_dl:
    outputs = model(xb)
    _, predicted = torch.max(outputs.data, 1)
    total += yb.size(0)
    correct += (predicted == yb).sum().item()

print('验证集准确率: %d %%' % (100 * correct / total))
```


# 第三节    神经网络实现回归任务

#### 气温预测回归

```python
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import torch.optim as optim

features = pd.read_csv('temps.csv')


```
